In [13]:
import json
import requests
import urllib.request
import pandas as pd
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_rows', 100)
#import networx as nx
#from pyvis.network import Network
import ast
from IPython.display import HTML

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")
import nlpsparqlclient as nlpsc

## NLP NER/RR

In [7]:
search_text = "Which Adverse Events produced by drugs for Metastatic Cancer for the last 3 years in the US"

entities, relations = nlpsc.start_nlp_client(search_text)

HTTPError: 400 Client Error: BAD REQUEST for url: http://localhost:5000/nlpservice?search=Which+Adverse+Events+produced+by+drugs+for+Metastatic+Cancer+for+the+last+3+years+in+the+US

In [8]:
def remove_plural(s):
    if s[-1] == 's':
        s = s.rstrip(s[-1])
    return s

## Ontology

In [10]:
def check4relationship(entities):
    # given 2 entities, see if a relation exists between them in the ontology
    relation_dict = {}
    data = []
    if len(entities) < 2:
        result = []
    else:
        for e1 in entities:
            for e2 in entities:
                if e1 != e2:
                    result = nlpsc.start_predicate_client(e1, e2)

                    if result is not None:
                        relation_dict = {'entity1': remove_plural(e1), 'relation': result['data'][0], 'entity2': remove_plural(e2)}
                        data.append(relation_dict)
        if data != []:
            return pd.DataFrame(data=data)
        else:
            return("There are no relationships")
                                     

In [ ]:
rel_data_df = check4relationship(entities)
rel_data_df_dict = rel_data_df.to_dict()
rel_data_df

In [11]:
def check4triples(entities, relations):
    # check for existing triples in the ontology
    triple_dict = {}
    data = []
    for entity1 in entities:
        for relation in relations:
            for entity2 in entities:

                if results['data']:
                    triple_dict = {'entity1': remove_plural(entity1), 'relation': relation, 'entity2': remove_plural(entity2)}
                    data.append(triple_dict)
    if data != []:
        return pd.DataFrame(data=data)
    else:
        return("There are no triples")


In [ ]:
df = check4triples(entities, relations)
df_dict = df.to_dict()

In [ ]:
df

In [12]:
def check4individuals(entity, relation):
    # find any individuals from a given entity in the ontology
    results = nlpsc.start_individuals_client(entity, relation)

    if results != []:
        if results['data'] != '[]':
            return results
    else:
        return("There are no individuals for " + entity)


In [ ]:
individuals = []
for e in entities:
    for r in relations:
        indiv_dict = check4individuals(e, r)
        if indiv_dict != None:
            for k,v in indiv_dict.items():
                individuals = ast.literal_eval(v)

## Results to user search query

In [ ]:
df_html = pd.DataFrame(individuals, columns=['Adverse Events'])
df_html.to_csv(df_html.columns[0].replace(' ', '_') + '.csv', index=False)
HTML(df_html.to_html(index=False))

## Knowledge Graph

In [ ]:
net =  Network(height='900px', width='100%', directed=True, notebook=True, heading='Clinical Trials')
net.barnes_hut(central_gravity=1.7, spring_length=70)
net.show_buttons(filter_=['physics'])

In [ ]:
triples = [v[0] for k,v in df_dict.items()]

i = 0
for node in triples:
    if i == 0:
        color = 'teal'
    else:
        color = 'aqua'

    if i != 1:
        net.add_node(node, size=5-, shape='circle', title=node, label=node, color=color)
    else:
        title = node
        label = node
    i+=1
net.add_edge(net.nodes[0]['id'], net.nodes[1]['id'], width=2, title=title, label=label, color='#ff0000')

In [ ]:
for node in individuals:
    net.add_node(node, title=node, label=node, color='orange')
    net.add_edge(node, net.nodes[0]['id'], width=2, title='a kind of', label='a kind of', color='purple')

In [ ]:
net.show('basic.html')

In [ ]:
net.save_graph('Adverse_Events.html')